In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

from browsermobproxy import Server
import pandas as pd


In [2]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://longportapp.com/en/markets/")

print(driver.title)
driver.page_source

# driver.close()
driver.quit()

LongPort | Research and trade with high-value investors


In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import undetected_chromedriver as uc

# options = Options()
options = uc.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--auto-open-devtools-for-tabs")
driver = uc.Chrome(options=options)
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get("https://longportapp.com/en/markets/")
print(driver.title)
driver.quit()

LongPort | Research and trade with high-value investors


In [6]:
import os
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Directory to save the downloaded files
download_dir = "/home/disk1/finance_quant/efinance/downloaded_files"
os.makedirs(download_dir, exist_ok=True)

# Initialize ChromeDriver
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

# Enable logging
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

driver = webdriver.Chrome(options=options)

# Function to save downloaded file (if applicable)
def save_file(url, content):
    filename = os.path.join(download_dir, url.split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(content.encode('utf-8'))
    print(f"Downloaded {filename}")
    
def save_json(content):
  filename = os.path.join(download_dir, "data.json")
  with open(filename, "w") as outfile:
    outfile.write(content)

# Function to process performance logs
def process_browser_logs_for_network_events(logs):
    for entry in logs:
        log = json.loads(entry['message'])['message']
        if (
            'Network.responseReceived' in log['method']
            and 'params' in log
            and 'response' in log['params']
        ):
            response = log['params']['response']
            url = response['url']
            if any(url.endswith(ext) for ext in ['.js', '.css', '.json', '.svg']):
                try:
                    content = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': log['params']['requestId']})
                    save_file(url, content['body'])
                except Exception as e:
                    print(f"Error getting response body for {url}: {e}")
                    
# Function to process performance logs
def process_browser_logs_for_network_events_json(logs):
    target_url = "https://api.lbkrs.com/market/concept_stocks"
    
    for entry in logs:
        log = json.loads(entry['message'])['message']
        if (
            'Network.responseReceived' in log['method']
            and 'params' in log
            and 'response' in log['params']
        ):
            response = log['params']['response']
            url = response['url']
            if url.startswith(target_url):
                try:
                    content = driver.execute_cdp_cmd('Network.getResponseBody', {'requestId': log['params']['requestId']})
                    # save_json(json.loads(content['body']))
                    print(content['body'])
                    return True  # Found and saved the target URL
                except Exception as e:
                    print(f"Error getting response body for {url}: {e}")
    
    return False  # Target URL not found

try:
    # Enable network interception
    driver.execute_cdp_cmd('Network.enable', {})

    # Open a webpage
    driver.get("https://longportapp.com/en/markets/")

    # Wait for page to load (adjust wait time and selector as needed)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "portal")))

    # Add a sleep to allow time for resources to be captured
    import time
    time.sleep(5)

    # Retrieve and process performance logs
    logs = driver.get_log('performance')
    process_browser_logs_for_network_events_json(logs)

finally:
    # Quit the driver
    driver.quit()

Error getting response body for https://api.lbkrs.com/market/concept_stocks?concept_index=BK%2FHK%2FIN20211&indicator=1&limit=50&offset=0&order=0&tab_key=0&filter_tag_key=3: Message: unknown error: unhandled inspector error: {"code":-32000,"message":"No resource with given identifier found"}
  (Session info: chrome-headless-shell=126.0.6478.126)
Stacktrace:
#0 0x5586860c5c7a <unknown>
#1 0x558685da8e2c <unknown>
#2 0x558685d8f446 <unknown>
#3 0x558685d8db48 <unknown>
#4 0x558685d8e33f <unknown>
#5 0x558685d8e294 <unknown>
#6 0x558685dabe4f <unknown>
#7 0x558685e4446a <unknown>
#8 0x558685e185c2 <unknown>
#9 0x558685e37303 <unknown>
#10 0x558685e18363 <unknown>
#11 0x558685de8247 <unknown>
#12 0x558685de8b9e <unknown>
#13 0x55868608c24b <unknown>
#14 0x5586860902f1 <unknown>
#15 0x558686077afe <unknown>
#16 0x558686090e52 <unknown>
#17 0x55868605c79f <unknown>
#18 0x5586860b5638 <unknown>
#19 0x5586860b5810 <unknown>
#20 0x5586860c4dac <unknown>
#21 0x7f1085d4b609 start_thread

Error ge

In [3]:
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

# ... (previous code remains the same)

# Function to download specific URL
def download_specific_url(driver, url):
    try:
        # Get the cookies from the driver
        cookies = driver.get_cookies()
        
        # Convert cookies to a dictionary
        cookie_dict = {cookie['name']: cookie['value'] for cookie in cookies}
        
        # Get headers from the browser
        headers = {
            'User-Agent': driver.execute_script("return navigator.userAgent;"),
            'Referer': 'https://longportapp.com/',
            'Origin': 'https://longportapp.com'
        }
        
        # Make the request with cookies and headers
        response = requests.get(url, cookies=cookie_dict, headers=headers)
        response.raise_for_status()
        content = response.json()
        save_file(url, content, is_json=True)
    except Exception as e:
        print(f"Error downloading {url}: {e}")

# ... (in the try block, after navigating to the page)

# Enable Performance Logging
driver.execute_cdp_cmd("Network.enable", {})
driver.execute_cdp_cmd("Performance.enable", {})

# Navigate to the main page first
driver.get("https://longportapp.com/en/markets/")

# Wait for page to load (adjust wait time and selector as needed)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "portal")))

# Add a sleep to allow time for resources to be captured
import time
time.sleep(5)

# Download specific URL
specific_url = "https://api.lbkrs.com/market/concept_stocks?concept_index=BK%2FHK%2FIN20263&indicator=1&limit=1&offset=0&order=0&tab_key=0&filter_tag_key=3"
download_specific_url(driver, specific_url)

# ... (rest of the code remains the same)

MaxRetryError: HTTPConnectionPool(host='localhost', port=39273): Max retries exceeded with url: /session/577f27b226cea7d844f3c3d8288ec75c/goog/cdp/execute (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa27c0c35b0>: Failed to establish a new connection: [Errno 111] Connection refused'))

driver.get_page